In [1]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import ipynbname
import datetime
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer, MinMaxScaler

from pytorch_tabular import TabularModel
from pytorch_tabular.models import NodeConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)

from Process_Function import RareCategoryTransformer
from Visualization_function import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore')

## 2. Data Load

In [2]:
train_path = './data/train.csv'
test_path = './data/test.csv'
sample_path = './data/sample_submission.csv'

# 학습/평가 데이터 로드
train = pd.read_csv(train_path).drop(columns=['UID'])
test = pd.read_csv(test_path).drop(columns=['UID'])

print(train.shape, test.shape)

train['채무 불이행 여부'].value_counts(normalize=True)

(10000, 17) (2062, 16)


채무 불이행 여부
0    0.6588
1    0.3412
Name: proportion, dtype: float64

In [3]:
train

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,주택 담보 대출 (거주 중),1339473.0,10년 이상,0.0,9,18.7,319027.5,0,68,0,부채 통합,단기 상환,126216.0,177028.5,6237.0,755,0
9996,주택 담보 대출 (거주 중),2297230.5,2년,0.0,11,28.3,399799.5,0,7,0,주택 개보수,장기 상환,371907.0,347449.5,53301.0,707,0
9997,주택 담보 대출 (거주 중),1221523.5,10년 이상,0.0,9,30.1,823305.0,0,14,0,부채 통합,장기 상환,869736.0,176905.5,11436.0,733,0
9998,자가,3343584.0,10년 이상,0.0,10,20.3,724314.0,0,25,0,부채 통합,단기 상환,443008.5,139294.5,25567.5,696,0


In [4]:
discrete_cols = [
    '주거 형태',
    '현재 직장 근속 연수',
    '대출 목적',
    '대출 상환 기간',
]

num_cols = [
    '연간 소득',
    '체납 세금 압류 횟수',
    '개설된 신용계좌 수',
    '신용 거래 연수',
    '최대 신용한도',
    '신용 문제 발생 횟수',
    '마지막 연체 이후 경과 개월 수',
    '개인 파산 횟수',
    '현재 대출 잔액',
    '현재 미상환 신용액',
    '월 상환 부채액',
    '신용 점수',
]

target_cos = '채무 불이행 여부'

In [5]:
# visualize_with_target(train, discrete_cols, threshold=1, target_column='채무 불이행 여부')

In [6]:
# visualize_continuous_with_target(train, num_cols, target_column='채무 불이행 여부')

In [7]:
def 현재직장근속연수(train_df, test_df):
    # mapping = {
    #     '1년 미만': 0.5,
    #     '1년': 1,
    #     '2년': 2,
    #     '3년': 3,
    #     '4년': 4,
    #     '5년': 5,
    #     '6년': 6,
    #     '7년': 7,
    #     '8년': 8,
    #     '9년': 9,
    #     '10년 이상': 10
    # }

    mapping = {
        '1년 미만': 1,
        '1년': 1,
        '2년': 2,
        '3년': 3,
        '4년': 3,
        '5년': 5,
        '6년': 6,
        '7년': 6,
        '8년': 8,
        '9년': 8,
        '10년 이상': 10
    }

    train_df['현재 직장 근속 연수'] = train_df['현재 직장 근속 연수'].map(mapping).astype(int)
    test_df['현재 직장 근속 연수'] = test_df['현재 직장 근속 연수'].map(mapping).astype(int)

    return train_df, test_df

def RareCategory_to_others(train, test, cols, threshold=10):
    transformer = RareCategoryTransformer(threshold=threshold, columns=cols)
    train = transformer.fit_transform(train)
    test = transformer.transform(test)
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def num_feature_scailing(train, test):
    cols_to_divide = [
        '연간 소득',
        '최대 신용한도',
        '현재 대출 잔액',
        '현재 미상환 신용액',
        '월 상환 부채액',
    ]
    train[cols_to_divide] = train[cols_to_divide] / 100000
    test[cols_to_divide] = test[cols_to_divide] / 100000

    cols_to_log = [
        '연간 소득',
        '최대 신용한도',
        '현재 대출 잔액',
        '현재 미상환 신용액',
        '월 상환 부채액',
    ]
    log_transformer = FunctionTransformer(np.log1p, validate=True)
    train[cols_to_log] = log_transformer.fit_transform(train[cols_to_log])
    test[cols_to_log] = log_transformer.transform(test[cols_to_log])

    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '채무 불이행 여부'
    ]
    scaler = StandardScaler()
    train[cols_to_scale] = scaler.fit_transform(train[cols_to_scale])
    test[cols_to_scale] = scaler.transform(test[cols_to_scale])

    return train, test

In [8]:
# 실험 내용
experiment_desc = '''
NODE
'''

In [9]:
def all_process(train, test):
    train, test = 현재직장근속연수(train, test)
    cat_cols = [
        '주거 형태',
        '현재 직장 근속 연수',
        '대출 목적',
        '대출 상환 기간',
    ]
    train, test = RareCategory_to_others(train, test, cols=cat_cols, threshold=5)
    train, test = type_to_category(train, test, cat_cols)

    train, test = num_feature_scailing(train, test)

    return train, test

train = pd.read_csv(train_path).drop(columns=['UID'])
test = pd.read_csv(test_path).drop(columns=['UID'])

train, test = all_process(train, test)

train.shape

(10000, 17)

In [10]:
train

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,0.068020,10,-0.279027,-0.703173,-0.899120,-0.577705,-0.507403,-0.344278,0.742870,부채 통합,단기 상환,-0.231827,-0.255114,-0.945892,0.399787,0
1,월세,0.105453,10,-0.279027,-1.568910,-0.663217,-0.671003,-0.507403,-0.993935,-0.442309,부채 통합,단기 상환,1.526475,-1.368039,0.240291,0.399787,0
2,월세,-0.614271,3,-0.279027,-0.053870,-0.149779,-0.389856,0.302886,2.154402,3.113228,부채 통합,단기 상환,-1.109930,0.710818,-0.692746,0.978807,1
3,월세,-1.093273,6,-0.279027,0.595432,-0.704847,-0.553415,0.302886,-0.444225,0.742870,부채 통합,단기 상환,-0.958730,0.489423,0.693506,0.908623,1
4,월세,1.629181,2,-0.279027,-0.270305,0.863219,0.193862,-0.507403,0.055511,-0.442309,부채 통합,장기 상환,1.822863,-1.232496,-1.180968,0.119050,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,주택 담보 대출 (거주 중),-0.637868,10,-0.279027,-0.703173,-0.163656,-0.777399,-0.507403,1.854561,-0.442309,부채 통합,단기 상환,-1.916114,-0.518485,-1.150669,0.189234,0
9996,주택 담보 대출 (거주 중),0.392765,2,-0.279027,-0.270305,1.168506,-0.579497,-0.507403,-1.193830,-0.442309,주택 개보수,장기 상환,-0.317622,0.265266,1.991681,-0.652977,0
9997,주택 담보 대출 (거주 중),-0.810621,10,-0.279027,-0.703173,1.418286,0.109576,-0.507403,-0.844014,-0.442309,부채 통합,장기 상환,1.248153,-0.519211,-0.741281,-0.196779,0
9998,자가,1.124660,10,-0.279027,-0.486739,0.058371,-0.017750,-0.507403,-0.294305,-0.442309,부채 통합,단기 상환,-0.012522,-0.757848,0.281743,-0.845983,0


In [11]:
cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
numeric_cols = [
    col for col in numeric_cols
    if col not in cat_cols and col != '채무 불이행 여부'
]


data_config = DataConfig(
    target=[
        "채무 불이행 여부"
    ],  # target should always be a list.
    continuous_cols=numeric_cols,
    categorical_cols=cat_cols,
)

trainer_config = TrainerConfig(
    # auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=128,
    data_aware_init_batch_size=2000,
    max_epochs=100,
    accelerator='gpu',
    early_stopping='valid_loss',
    early_stopping_min_delta=0.001,
    early_stopping_mode='min',
    early_stopping_patience=3,
    checkpoints='valid_loss',
    checkpoints_path='saved_models',
    checkpoints_save_top_k=1,
    progress_bar='none',
)
optimizer_config = OptimizerConfig()

In [21]:
n_splits = 10
# seed_list = [42, 333, 777, 2024, 77]
seed_list = [777]
all_auc = []
test_preds = []
is_first = True

for seed in seed_list:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    auc_scores = []
    fold_test_preds = []

    train = pd.read_csv(train_path).drop(columns=['UID'])
    test = pd.read_csv(test_path).drop(columns=['UID'])

    for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['채무 불이행 여부'])):
        fold_train, fold_valid = train.iloc[train_idx].copy().reset_index(drop=True), train.iloc[valid_idx].copy().reset_index(drop=True)
        fold_train2 = fold_train.copy()
        fold_test = test.copy()

        fold_train, fold_valid = all_process(fold_train, fold_valid)
        fold_train2, fold_test = all_process(fold_train2, fold_test)

        model_config = NodeConfig(
            task="classification",
            learning_rate=0.001,
            embedding_dims=None,
            embedding_dropout=0.0,
            num_layers=1,
            num_trees=2048,
            additional_tree_output_dim=3,
            depth=6,
            choice_function='entmax15',
            bin_function='entmoid15',
            max_features=None,
            input_dropout=0.0,
            seed=seed,
        )

        tabular_model = TabularModel(
            data_config=data_config,
            model_config=model_config,
            optimizer_config=optimizer_config,
            trainer_config=trainer_config,
            verbose=False,
        )

        tabular_model.fit(train=fold_train, validation=fold_valid, seed=seed)

        valid_preds = tabular_model.predict(fold_valid).iloc[:, 1]
        fold_auc = roc_auc_score(fold_valid['채무 불이행 여부'], valid_preds)

        auc_scores.append(fold_auc)
        test_pred = tabular_model.predict(fold_test).iloc[:, 1]
        fold_test_preds.append(test_pred)

    test_preds.append(np.mean(fold_test_preds, axis=0))

    # 각 seed별 평균 AUC와 표준편차 출력
    seed_auc_mean = np.mean(auc_scores)
    seed_auc_std = np.std(auc_scores)
    all_auc.append(seed_auc_mean)
    print(f"Seed {seed} - Average AUC: {seed_auc_mean:.5f} (STD: {seed_auc_std:.5f})")
    print('=' * 60)



# 전체 결과에 대한 평균 및 표준편차 출력
total_auc_mean = np.mean(all_auc)
total_auc_std = np.std(all_auc)
print('-' * 60)
print(f'Total Average AUC: {total_auc_mean:.6f} (STD: {total_auc_std:.6f})')

Seed set to 777


2025-03-11 22:53:44,804 - {pytorch_tabular.models.node.node_model:74} - INFO - Data Aware Initialization of NODE   
using a forward pass with 2000 batch size....

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | _backbone        | NODEBackbone     | 1.0 M  | train
1 | _embedding_layer | Embedding1dLayer | 175    | train
2 | _head            | Lambda           | 0      | train
3 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
1.0 M     Trainable params
769       Non-trainable params
1.0 M     Total params
4.100     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
Seed set to 777


2025-03-11 22:55:04,897 - {pytorch_tabular.models.node.node_model:74} - INFO - Data Aware Initialization of NODE   
using a forward pass with 2000 batch size....

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | _backbone        | NODEBackbone     | 1.0 M  | train
1 | _embedding_layer | Embedding1dLayer | 175    | train
2 | _head            | Lambda           | 0      | train
3 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
1.0 M     Trainable params
769       Non-trainable params
1.0 M     Total params
4.100     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
Seed set to 777


...

2025-03-11 23:02:16,537 - {pytorch_tabular.models.node.node_model:74} - INFO - Data Aware Initialization of NODE   
using a forward pass with 2000 batch size....

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | _backbone        | NODEBackbone     | 1.0 M  | train
1 | _embedding_layer | Embedding1dLayer | 168    | train
2 | _head            | Lambda           | 0      | train
3 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
1.0 M     Trainable params
769       Non-trainable params
1.0 M     Total params
4.100     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
Seed set to 777


2025-03-11 23:04:06,733 - {pytorch_tabular.models.node.node_model:74} - INFO - Data Aware Initialization of NODE   
using a forward pass with 2000 batch size....

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | _backbone        | NODEBackbone     | 1.0 M  | train
1 | _embedding_layer | Embedding1dLayer | 175    | train
2 | _head            | Lambda           | 0      | train
3 | loss             | CrossEntropyLoss | 0      | train
--------------------------------------------------------------
1.0 M     Trainable params
769       Non-trainable params
1.0 M     Total params
4.100     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Seed 777 - Average AUC: 0.73985 (STD: 0.02309)
------------------------------------------------------------
Total Average AUC: 0.739852 (STD: 0.000000)


In [23]:
old_auc = 0.744533 * 100
old_std = 0.001171 * 100

new_auc = total_auc_mean * 100
new_std = total_auc_std * 100

def calculate_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value) * 100 if old_value != 0 else float('inf')
    return change, percentage_change

def format_change(change):
    return f"{change:+.6f}"

# 각 지표의 변화량 계산
auc_change, auc_pct = calculate_change(old_auc, new_auc)
std_change, std_pct = calculate_change(old_std, new_std)

# 결과 출력
print("\n========== 모델 성능 변화 ==========")
print(f"{'Metric':<8}  {'AUC':>12}  {'Acc':>12}")
print("-" * 36)
print(f"{'Old':<8}  {old_auc:>12.6f}  {old_std:>12.6f}")
print(f"{'New':<8}  {new_auc:>12.6f}  {new_std:>12.6f}")
print(f"{'Change':<8}  {format_change(auc_change):>12}  {format_change(std_change):>12}")
print(f"{'% Change':<8}  {auc_pct:>11.4f}%  {std_pct:>11.4f}%")
print("=" * 36)


========== 모델 성능 변화 ==========
Metric             AUC           Acc
------------------------------------
Old          74.453300      0.117100
New          73.985216      0.000000
Change       -0.468084     -0.117100
% Change      -0.6287%    -100.0000%


In [24]:
sample_submission = pd.read_csv(sample_path)
sample_submission['채무 불이행 확률'] = np.mean(test_preds, axis=0)

ratio = train['채무 불이행 여부'].value_counts(normalize=True)[1]
real_true_count = int(ratio * len(sample_submission))
print(f'test의 True 갯수: {real_true_count:<5} (추정)')

count = (sample_submission['채무 불이행 확률'] >= 0.5).sum()
print(f'test의 True 갯수: {count:<5} (예측 결과)')

test의 True 갯수: 703   (추정)
test의 True 갯수: 238   (예측 결과)


In [25]:
now = datetime.datetime.now().strftime("%m%d_%H%M")

In [26]:
submission_path = 'Submission'
if not os.path.exists(submission_path):
    os.makedirs(submission_path)

code_dir = 'Code'
if not os.path.exists(code_dir):
    os.makedirs(code_dir)

submission_name = f"submission_{now}.csv"
new_notebook_name = f"code_{now}.ipynb"

sample_submission.to_csv(os.path.join(submission_path, submission_name), encoding='UTF-8-sig', index=False)

current_notebook = ipynbname.path()
new_notebook_path = os.path.join(code_dir, new_notebook_name)
shutil.copy(current_notebook, new_notebook_path)

print(f"Notebook saved in '{code_dir}' as '{new_notebook_name}'")

Notebook saved in 'Code' as 'code_0311_2305.ipynb'


In [27]:
# 📌 SQLite 데이터베이스 설정
db_path = "lgbm_experiment_results.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 📌 테이블 생성 (처음 실행 시)
cursor.execute('''
CREATE TABLE IF NOT EXISTS experiments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_name TEXT,
    experiment_desc TEXT,
    auc REAL,
    std REAL
)
''')

In [28]:
# 데이터 삽입
cursor.execute('''
INSERT INTO experiments (code_name, experiment_desc, auc, std)
VALUES (?, ?, ?, ?)
''', (new_notebook_name, experiment_desc.strip(), new_auc, new_std))

# 변경사항 저장 & 연결 종료
conn.commit()
conn.close()

print(f"Experiment '{new_notebook_name}' successfully saved in database")

Experiment 'code_0311_2305.ipynb' successfully saved in database


In [29]:
import sqlite3
import pandas as pd

# SQLite 데이터 조회 함수
def get_experiment_results(db_path="lgbm_experiment_results.db", num_results=10):
    """
    SQLite 데이터베이스에서 중복된 실험 데이터를 제거하고, 최근 num_results개의 실험 데이터를 불러오는 함수.
    Returns:
        - Pandas DataFrame: 중복 제거된 실험 데이터
    """
    conn = sqlite3.connect(db_path)

    # 중복 제거 & 최신 데이터 선택하는 SQL 쿼리
    query = f"""
    SELECT * FROM experiments
    WHERE id IN (
        SELECT MAX(id)  -- 가장 최신 데이터 선택
        FROM experiments
        GROUP BY code_name -- id 제외하고 중복 판단
    )
    ORDER BY id DESC  -- 최신 데이터부터 정렬
    LIMIT {num_results};
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    return df


df_results = get_experiment_results(num_results=100)
df_results.to_csv('lgbm_experiment_results.csv', index=False, encoding='utf-8-sig', float_format='%.4f')
df_results

,id,code_name,experiment_desc,auc,std
0,31,code_0311_2305.ipynb,NODE,73.985216,0.000000
1,30,code_0311_2246.ipynb,NODE,73.985216,0.000000
2,29,code_0311_2219.ipynb,NODE,73.816734,0.000000
3,28,code_0311_2039.ipynb,NODE,68.368928,0.000000
4,27,code_0310_2158.ipynb,TabPFN,74.776633,0.000000
5,26,code_0310_0048.ipynb,TabPFN,74.792289,0.000000
6,25,code_0309_2215.ipynb,TabPFN,74.813522,0.000000
7,24,code_0309_2212.ipynb,TabPFN,74.873422,0.033450
8,23,code_0306_2055.ipynb,"샘플링, 시드777, 캣부스트, 샘플 200개, 컬럼 랜덤 선택",74.588855,0.000000
9,22,code_0306_1946.ipynb,"샘플링, 시드777, 캣부스트, 샘플 200개, 컬럼 8개 고정",71.412381,0.000000
